In [1]:
!hdfs dfs -ls /user/hive/warehouse/

In [2]:
!pip show delta-spark

Name: delta-spark
Version: 4.0.0
Summary: Python APIs for using Delta Lake with Apache Spark
Home-page: https://github.com/delta-io/delta/
Author: The Delta Lake Project Authors
Author-email: delta-users@googlegroups.com
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: importlib-metadata, pyspark
Required-by: 


In [3]:
# 1.1. Importar tudo o que precisamos
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

# 1.2. Construir o SparkSession com Hive e Delta ativados
spark = (
    SparkSession.builder
    .appName("TesteSparkHiveDelta")
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive:9083")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .enableHiveSupport()
    .getOrCreate()
)

# 1.3. Verificar versão do Spark e se o Hive está disponível
print("Spark versão:", spark.version)
print("Suporte a Hive ativado?:", spark.conf.get("spark.sql.catalogImplementation"))  # deve imprimir "hive"

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark versão: 4.0.0
Suporte a Hive ativado?: hive


In [4]:
spark.sql("SHOW DATABASES").show()

25/06/10 10:26:39 WARN HiveConf: HiveConf of name hive.server2.thrift.jvm.args does not exist
25/06/10 10:26:39 WARN HiveConf: HiveConf of name hive.server2.thrift.java.port does not exist
Hive Session ID = 60c91b88-1870-4eb5-a7e8-11dc69e670de


+---------+
|namespace|
+---------+
|  default|
|sample_db|
| spark_db|
| teste_db|
+---------+



In [5]:
spark.sql("SHOW TABLES IN spark_db").show()

+---------+-------------------+-----------+
|namespace|          tableName|isTemporary|
+---------+-------------------+-----------+
| spark_db|tabela_hive_parquet|      false|
| spark_db|tabela_hive_externa|      false|
+---------+-------------------+-----------+



In [6]:
spark.sql("SELECT * FROM spark_db.tabela_hive_externa").show()

25/06/10 10:26:41 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: hdfs://namenode:9000/hdfs_exemplo_dir/tabela_externa_hive.
java.io.FileNotFoundException: File does not exist: hdfs://namenode:9000/hdfs_exemplo_dir/tabela_externa_hive
	at org.apache.hadoop.hdfs.DistributedFileSystem$29.doCall(DistributedFileSystem.java:1832)
	at org.apache.hadoop.hdfs.DistributedFileSystem$29.doCall(DistributedFileSystem.java:1825)
	at org.apache.hadoop.fs.FileSystemLinkResolver.resolve(FileSystemLinkResolver.java:81)
	at org.apache.hadoop.hdfs.DistributedFileSystem.getFileStatus(DistributedFileSystem.java:1840)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.hive.HiveMetastoreCatalog.$anonfun$convertToLogicalRelation$5(HiveMetastoreCatalog.scala:301)
	at scala.Option.getOrEls

AnalysisException: [PATH_NOT_FOUND] Path does not exist: hdfs://namenode:9000/hdfs_exemplo_dir/tabela_externa_hive. SQLSTATE: 42K03